In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
AUTOTUNE = tf.data.experimental.AUTOTUNE

import matplotlib.pyplot as plt

In [ ]:
BASE_DIR = '../input/cassava-leaf-disease-classification'
TRAIN_IMAGES_FOLDER = os.path.join(BASE_DIR, 'train_images')
TEST_IMAGES_FOLDER = os.path.join(BASE_DIR, 'test_images')

df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
df.image_id = BASE_DIR + '/train_images/' + df.image_id

In [ ]:
def data_pipeline(image_file):
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SHAPE)
    image = preprocess_input(image)
    return image

In [ ]:
def create_model(input_shape):
    base_model = EfficientNetB3(
        weights='imagenet',
        include_top=False,
        input_shape = tuple(input_shape + [3]))
    base_model.trainable = False

    x = base_model.output
    outputs = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
    return model

In [ ]:
IMG_SHAPE = [300, 300]
model = create_model(IMG_SHAPE)

dataset = tf.data.Dataset.from_tensor_slices(df.image_id)
dataset = dataset.map(data_pipeline, num_parallel_calls=AUTOTUNE)
dataset = dataset.batch(128)

features = model.predict(dataset)
np.save('features.npy', features)